# Ejemplo de Clustering con HDBSCAN

Un algoritmo de clustering espacial es DBSCAN, que está disponible en ``sklearn.cluster.DBSCAN``.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/a/af/DBSCAN-Illustration.svg/800px-DBSCAN-Illustration.svg.png)

Su nombre significa "barrido basado en densidad." Lo que hace DBSCAN es definir tres tipos de puntos en función de los otros puntos que lo rodean:

- _Núcleos_ (puntos rojos): conjuntos de puntos que, entre sí, están dentro de un radio de distancia y, en total, son más que una cierta cantidad especificada como hiperparámetro.
- _Puntos Alcanzables_ (puntos amarillos): puntos que no son núcleos, pero que están dentro del umbral de tolerancia de distancia a puntos núcleos.
- _Ruido_ (punto azul): puntos que no son alcanzables desde los núcleos.

Considerando eso, se vuelve intuitivo el hecho de que no necesitamos especificar el número de clusters, sino el umbral de distancia para la densidad, y, por tanto, que la forma de los clusters puede ser arbitraria. Al mismo tiempo, habrá puntos que no estarán agrupados en clusters.

Veremos un ejemplo de aplicar una versión de DBSCAN que está en el estado del arte, conocida como [HDBSCAN](https://hdbscan.readthedocs.io/en/latest/) (por Hierarchical DBSCAN). Para instalarla debemos utilizar el siguiente comando:

`pip install hdbscan`

Su API es la misma de Scikit-Learn, así que será sencillo familiarizarnos con su uso.

La diferencia entre HDBSCAN y DBSCAN es que HDBSCAN tiene una noción de distancia umbral flexible. Esto es útil cuando la densidad de los puntos varía y, por tanto, un único umbral podría llevar a resultados erróneos.

In [ ]:
# análisis
import pandas as pd
import geopandas as gpd
import numpy as np

# clustering
from hdbscan import HDBSCAN

# visualización
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patheffects as path_effects
from matplotlib.colors import rgb2hex

# Esto configura la apariencia de los gráficos utilizando configuraciones de seaborn
sns.set(context='notebook', style='ticks', palette='inferno', font='Linux Biolinum O', font_scale=1.1)

# Esto es una instrucción de Jupyter que hace que los gráficos se desplieguen en el notebook
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
import sys
from pathlib import Path

AVES_ROOT = Path("..") / ".." / ".."

EOD_PATH = AVES_ROOT / "data" / "external" / "EOD_STGO"
EOD_PATH

In [ ]:
zones = gpd.read_file(AVES_ROOT / "data" / "processed" / "scl_zonas_urbanas.json")
zones.plot()

In [ ]:
from aves.data import eod

hogares = eod.read_homes(EOD_PATH)
personas = eod.read_people(EOD_PATH).merge(hogares)

viajes = eod.read_trips(EOD_PATH).merge(personas).merge(hogares)

viajes["PesoLaboral"] = viajes["FactorLaboralNormal"] * viajes["Factor_LaboralNormal"]

viajes = viajes[pd.notnull(viajes["PesoLaboral"])]

viajes.columns

In [ ]:
sns.countplot(y=viajes['Proposito'], color='grey')
sns.despine()

In [ ]:
propositos = ['Al trabajo', 'Al estudio', 'Trámites', 'De salud', 'De compras', 'Recreación']

Visualicemos los destinos de esos viajes:

In [ ]:
from aves.features.geo import to_point_geodataframe

destinos_viajes = to_point_geodataframe(
    viajes, "DestinoCoordX", "DestinoCoordY", crs="epsg:5361"
).pipe(lambda x: gpd.sjoin(x.to_crs(zones.crs), zones, op="within"))

destinos_viajes.plot(marker=".", markersize=1, alpha=0.5)

In [ ]:
from aves.visualization.figures import small_multiples_from_geodataframe
from aves.visualization.maps import dot_map

fig, axes = small_multiples_from_geodataframe(zones, len(propositos), height=7, col_wrap=3)

for prop, ax in zip(propositos, axes.flatten()):
    zones.plot(facecolor="#efefef", edgecolor="none", ax=ax)
    dot_map(
        ax,
        destinos_viajes[destinos_viajes["Proposito"] == prop],
        size=10,
        scale=0.15,
        alpha=0.5,
    )
    ax.set_title(prop)

## Mini-Proyecto: ¿Cuántos Centros hay en la Ciudad?¿Qué los caracteriza?

Como vemos en las imágenes previas, viajes hay en toda la ciudad. Sin embargo, sabemos que no todos los sectores de la ciudad reciben afluencias grandes de personas.

El centro histórico de Santiago es sólo uno, pero lugares que concentran actividades hay varios. Entonces, ¿cuántos son? Saberlo nos permitiría apoyar la planificación de la red de transporte, el fomento de instalación en centros sub-desarrollados, o incluso la identificación de oportunidades para crear un nuevo centro.

Definiremos como _centro_ un espacio de la ciudad en el que se concentran actividades. Y, utilizando Machine Learning no supervisado, buscaremos una manera de responder la pregunta, **agrupando los destinos de los viajes como una señal de la afluencia que tiene un lugar**.

### Elección de Viajes a Considerar

No todos los viajes son iguales. Como vemos en el primer gráfico, los viajes de regreso a casa son los más frecuentes, y están dispersos por toda la ciudad. Lo que queremos son estudiar _las actividades que se hacen fuera de casa_.

Por simplicidad nos enfocaremos en las seis actividades que seleccionamos antes.

In [ ]:
analysis_trips = destinos_viajes[destinos_viajes["Proposito"].isin(propositos)].to_crs(
    "epsg:5361"
)
len(analysis_trips)

Creamos nuestra matriz de características: las posiciones de los viajes.

In [ ]:
X = np.vstack((analysis_trips.geometry.x, analysis_trips.geometry.y)).T
X

Realizamos el procedimiento estándar:

- Inicializar instancia del modelo (que ya importamos en el preámbulo) con sus hiperparámetros. Usaremos dos: 
  - `min_cluster_size`: cantidad mínima de viajes para considerar un cluster.
  - `min_samples`: cantidad mínima de puntos núcleo para considerar un cluster.
- Ajustar la matriz de características `X`
- Predecir el vector de etiquetas `y`

Los últimos dos pasos los podemos realizar en una sola llamada al método `fit_predict(X)`:

In [ ]:
dbscan = HDBSCAN(min_cluster_size=400, min_samples=25)
analysis_trips["cluster"] = dbscan.fit_predict(X)
cluster_ids = analysis_trips["cluster"].value_counts()
len(cluster_ids) - 1

Con esos hiperparámetros tenemos 9 clusters. Ésta es la cantidad de viajes que poseen:

In [ ]:
cluster_ids

Para identificar los clusters utilizaremos una paleta de colores conocida como `husl`, que varía el tono de un color a otro:

In [ ]:
colors = sns.color_palette('husl', n_colors=len(cluster_ids) - 1)
sns.palplot(colors)

Crearemos un diccionario para pintar los puntos de los clusters. Le asignaremos el color gris a los puntos de ruido:

In [ ]:
palette = dict(zip(range(len(cluster_ids) - 1), map(rgb2hex, colors)))
palette[-1] = '#afafaf'
palette

In [ ]:
cluster_centroids = {}

for cluster_id in cluster_ids.index:
    cluster_trips = analysis_trips[analysis_trips.cluster == cluster_id]

    cluster_centroids[cluster_id] = (
        cluster_trips.geometry.x.mean(),
        cluster_trips.geometry.y.mean(),
    )

cluster_centroids

In [ ]:
zones_m = zones.to_crs('epsg:5361')
fig, ax = small_multiples_from_geodataframe(zones_m, 1, height=12)

zones_m.plot(ax=ax, edgecolor="grey", facecolor="#efefef")

# pintamos los puntitos de cada cluster
for cluster_id in sorted(cluster_ids.index):
    # viajes correspondientes a cada cluster
    cluster_trips = analysis_trips[analysis_trips.cluster == cluster_id]

    # dibujamos directamente con matplotlib
    cluster_trips.plot(ax=ax, alpha=0.8, markersize=2, color=palette[cluster_id])

    # para los clusters, agregamos la etiqueta correspondiente para poder identificarlos
    # pondremos la etiqueta en el promedio de las posiciones que tiene cada cluster
    if cluster_id >= 0:
        t = ax.text(
            cluster_centroids[cluster_id][0],
            cluster_centroids[cluster_id][1],
            str(cluster_id),
            horizontalalignment="center",
            fontsize=18,
            fontweight="bold",
            color="white",
        )

        # éste es un efecto gráfico que facilita la comprensión del texto
        t.set_path_effects(
            [
                path_effects.Stroke(linewidth=2, foreground="black"),
                path_effects.Normal(),
            ]
        )


¿Qué les parecen los resultados?

Consideraciones:

- El centro histórico está correctamente identificado.
- El eje Providencia aparece como un gran centro, y parte de Vitacura y Las Condes también. Esto es coherente con los distritos comerciales y de negocios que hay en el sector.
- ¿Maipú y San Bernardo casi completas son centros? Seguramente hay que ajustar algo, porque si bien podrían serlo (sobretodo el centro de Maipú), su extensión no es tan larga.
- Plaza de Puente Alto y Concha y Toro: esto sí tiene sentido, considerando la cantidad de lugares de trabajo y servicios que hay en esos lugares.
- Vicuña Mackenna y Avenida La Florida: ídem a lo anterior.
- Gran Avenida: también, sobretodo considerando el polo comercial que es.

Hay que jugar con los hiperparámetros para entender cómo se comporta el algoritmo, y así poder ajustarlos para obtener una mejor respuesta. Verán que algunos parámetros son más sensibles que otros, y pueden introducir grandes cambios en los resultados.

De momento, asumamos que estamos contentos con la respuesta.

### ¿Qué sigue?

El siguiente paso es buscar comprender qué caracteriza a cada uno de estos clusters.

Por ejemplo, podríamos preguntarnos cómo llega la gente a estos lugares. La encuesta tiene un atributo llamado `ModoDifusion` que indica el o los modos de transporte utilizados en el viaje:

In [ ]:
sns.countplot(y=analysis_trips['ModoDifusion'], color='grey')
sns.despine()

Entonces podemos hacer ese cálculo por cada modo (o combinación). Para ello utilizaremos la operación `groupby`:

In [ ]:
from aves.features.utils import normalize_rows

mot_table = (
    analysis_trips[analysis_trips["cluster"] >= 0]
    .drop_duplicates(subset=['Persona'], keep='first')
    .groupby(["cluster", "ModoDifusion"])["PesoLaboral"]
    .sum()
    .unstack()
    .fillna(0)
    .pipe(normalize_rows)
)
mot_table

Podemos visualizar el resultado con un `HeatMap`:

In [ ]:
plt.figure(figsize=(10, 7))
sns.heatmap(mot_table, annot=True, linewidth=1)

Una manera de facilitar nuestro análisis es ordenar la tabla de acuerdo a la similitud de uso de modos de transporte y de distribuciones por cluster. Esto lo podemos hacer con el método `sns.clustermap` (y aprovechemos de usar una escala más fácil de ver):

In [ ]:
sns.clustermap(
    mot_table, annot=True, linewidth=1, figsize=(12, 9), cmap="inferno_r", method="ward"
)

In [ ]:
mode_map = {
    'Bicicleta': 'Activo',
    'Taxi': 'Taxi',
    'Bip! - Otros Privado': 'Bip!',
    'Bip! - Otros Público': 'Bip!',
    'Caminata': 'Activo',
    'Otros': 'Auto',
    'Taxi Colectivo': 'Taxi',
    'Bip!': 'Bip!',
    'Auto': 'Auto'
}

analysis_trips['ModoSimple'] = analysis_trips['ModoDifusion'].map(mode_map)

In [ ]:
simple_mot_table = (
    analysis_trips[analysis_trips["cluster"] >= 0]
    .groupby(["cluster", "ModoSimple"])["PesoLaboral"]
    .sum()
    .unstack()
    .fillna(0)
    .pipe(normalize_rows)
)
sns.heatmap(simple_mot_table, annot=True, linewidth=1)

In [ ]:
mot_order = simple_mot_table.mean().sort_values(ascending=False)
mot_order

In [ ]:
from aves.visualization.figures import figure_from_geodataframe
from aves.visualization.colors import categorical_color_legend

fig, ax = small_multiples_from_geodataframe(zones_m, 1, height=12)

zones_m.plot(ax=ax, edgecolor="grey", facecolor="#efefef")

# pintamos los puntitos de cada cluster
for cluster_id in sorted(cluster_ids.index):
    if cluster_id < 0:
        continue
    # viajes correspondientes a cada cluster
    cluster_trips = analysis_trips[analysis_trips.cluster == cluster_id]

    # dibujamos directamente con matplotlib
    cluster_trips.plot(ax=ax, alpha=0.8, markersize=2, color=palette[cluster_id])

    # posición en el gráfico (coordenadas absolutas)
    pos = cluster_centroids[cluster_id]
    p = ax.transData.transform_point(pos)
    # posición en la figura (coordenadas relativas)
    p = fig.transFigure.inverted().transform_point(p)

    pie_size = 0.04
    pie_bounds = [p[0] - pie_size * 0.5, p[1] - pie_size * 0.5, pie_size, pie_size]

    box_inset = fig.add_axes(pie_bounds, label=cluster_id)
    box_inset.pie(
        simple_mot_table[mot_order.index].loc[cluster_id].values,
        wedgeprops=dict(edgecolor="black", linewidth=0.5),
        colors=colors, startangle=90
    )

    pos_y = 1.0
    va = "bottom"

    t = box_inset.annotate(
        cluster_id,
        (0.5, pos_y),
        xycoords="axes fraction",
        horizontalalignment="center",
        va=va,
        fontsize=8,
        fontweight="bold",
        color="white",
    )
    t.set_path_effects(
        [path_effects.Stroke(linewidth=2, foreground="black"), path_effects.Normal()]
    )

categorical_color_legend(ax, colors, mot_order.index, loc='center left')

### ¿Qué más hacer?

Otra opción es tratar de buscar maneras de evaluar los resultados. Por ejemplo, comparando los clusters obtenidos con el uso de suelo del Servicio de Impuestos Internos, o los planes reguladores de cada comuna. ¿Coinciden los bordes obtenidos con los definidos por las autoridades? ¿O el uso de la ciudad se desmarca de los bordes impuestos administrativamente?

Una manera podría ser visualizar dónde vive la gente que trabaja en cada cluster:

In [ ]:
origenes_viajes = to_point_geodataframe(
    analysis_trips[analysis_trips['cluster'] >= 0], "OrigenCoordX", "OrigenCoordY", "epsg:5361"
)

origenes_viajes.plot()

In [ ]:
fig, axes = small_multiples_from_geodataframe(
    zones_m, len(cluster_ids) - 1, height=7, col_wrap=4
)

for cluster_id, ax in zip(range(cluster_ids.index.max() + 1), axes.flatten()):
    if cluster_id < 0:
        continue
    zones_m.plot(facecolor="#efefef", edgecolor="none", ax=ax)
    origenes_viajes[origenes_viajes["cluster"] == cluster_id].plot(ax=ax, markersize=2, marker='.')
    analysis_trips[analysis_trips['cluster'] == cluster_id].plot(ax=ax, color=colors[cluster_id], markersize=2, marker='.')
    ax.set_title(cluster_id)

Como ven, es un algoritmo fácil de usar, y que, al mismo tiempo, tiene parámetros _algo_ intuitivos. 

Sin embargo, al igual que en todo otro modelo, ajustar los hiperparámetros requiere un trabajo constance y consciente. 

Y por supuesto también tiene limitaciones. Los datos que hemos utilizado son bidimensionales en lo que respecta al clustering. ¿Cómo podemos incorporar más variables? ¿Qué pasa con las funciones de distancia cuando hay más dimensiones involucradas? ¿Qué pasa cuando las dimensiones no tienen la misma semántica? Por ejemplo, uno pudiese querer clusterizar considerando también el ingreso de las personas. 

En conjunto con las preguntas exploratorias propuestas, entender las limitaciones y ventajas de un modelo son claves a la hora de decidir cuál utilizar. 
